# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

In [ ]:
!pip install -U --quiet lancedb pandas pydantic ##[This has been pre-installed for you]
!pip install pylance


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [4]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [ ]:
import lancedb

data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

# connect to a lancedb in ~/.lancedb
db = lancedb.connect("~/.lancedb")

# create a cats_and_dogs table using the Pydantic model as schema
db.create_table("cats_and_dogs", schema=CatsAndDogs)


Let's add some data

First some cats

In [12]:
table = db["cats_and_dogs"]

data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [16]:
table.add(data)

AddResult(version=2)

Let's preview the data

In [17]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [18]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [19]:
table.add(data)

AddResult(version=3)

In [20]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

**HINT** you'll need to use the `search` API for LanceTable and `limit` / `to_df` APIs. For examples you can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

In [ ]:
table.search([10.5, 10.]).limit(1).to_pandas()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.01


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

**HINT** you can add a call to `metric` in the call chain

In [ ]:
table.search([10.5, 10.]).metric('cosine').limit(1).to_pandas()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

In [28]:
table.search([10.5, 10.]).metric('cosine').where("species='cat'").limit(3).to_pandas()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862
1,"[-1.0, 0.0]",cat,himalayan,9.5,1.724138


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the vec_to_table function to convert that in to an Arrow table, which can then be added to the table.

In [10]:
%pip install pylance

from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(10000000, 16)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

/Users/rafael.esquicato/Documents/projects/udacity-genai-workspace/venv/lib/python3.12/site-packages/lancedb/__init__.py:238: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/Users/rafael.esquicato/Documents/projects/udacity-genai-workspace/venv/lib/python3.12/site-packages/lance/__init__.py:187: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [ ]:
query = np.random.randn(16)
table.search(query).limit(10).to_df()

/var/folders/2z/b4b21nyd0jxcp1kh766nw8hm0000gp/T/ipykernel_35202/229240492.py:2: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[-1.833849, -1.0171735, -0.29579082, 0.2335849...",5.429476
1,"[-1.8443601, -1.0400679, 0.3161163, 0.5617553,...",6.043387
2,"[-1.6019119, -0.39263755, -0.37030053, -0.0799...",6.257985
3,"[-1.3764764, -0.8535481, 0.22383904, 0.3510911...",6.837949
4,"[-2.4936032, -0.6832193, -0.74022067, 0.868196...",6.953854
5,"[-1.63474, 0.2712642, -1.4536238, 0.08773638, ...",7.103565
6,"[-1.6232439, 0.27233407, -1.814078, 1.6719476,...",7.134729
7,"[-1.6493179, -0.34854478, -0.6199672, 0.296823...",7.310455
8,"[-0.85379064, -1.3904358, -0.5101544, 1.121507...",7.321698
9,"[-0.854821, -0.43497977, -0.11106552, -0.04945...",7.347158


Please write code to compute the average latency of this query

In [ ]:
import time
latencies = []
for _ in range(10):
    start = time.perf_counter()
    table.search(query).limit(10).to_df()
    end = time.perf_counter()
    latencies.append(end - start)
print(f"Average latency: {sum(latencies) / len(latencies)} seconds")

/var/folders/2z/b4b21nyd0jxcp1kh766nw8hm0000gp/T/ipykernel_35202/1274102172.py:5: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


Average latency: 0.1679095290000987 seconds


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT** 
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector
3. This step can take about 7-10 minutes to process and execute in the classroom workspace.

In [16]:
## ### Now let's create an index
table.create_index(
    num_partitions=4000,
    num_sub_vectors=8,
    metric="cosine",
    index_type="IVF_PQ"
)

Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [17]:
table.search(query).limit(10).to_df()

/var/folders/2z/b4b21nyd0jxcp1kh766nw8hm0000gp/T/ipykernel_35202/4174343316.py:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[-1.8422635, 0.06667527, -0.35467818, 0.337573...",0.147348
1,"[-1.4197648, -1.2105553, -1.1874876, -0.070680...",0.177031
2,"[-2.1845164, -0.5901681, -0.72581893, 0.795833...",0.191785
3,"[-1.3108419, -1.0486406, -0.28533497, 0.849480...",0.200159
4,"[-1.597465, -1.7980316, -1.0044398, 1.0841285,...",0.201663
5,"[-2.6543875, -1.1256112, 0.31859818, 0.678983,...",0.207273
6,"[-1.2749931, -0.5393636, -0.92393786, -0.05325...",0.216797
7,"[-2.7982686, 0.19703262, -0.025551824, -0.2814...",0.220802
8,"[-2.181138, -0.6854764, -0.5358109, -0.6385386...",0.222463
9,"[-2.974352, -1.3004088, -0.81101286, 0.0237345...",0.223431


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [18]:
import time
latencies = []
for _ in range(10):
    start = time.perf_counter()
    table.search(query).limit(10).to_df()
    end = time.perf_counter()
    latencies.append(end - start)
print(f"Average latency: {sum(latencies) / len(latencies)} seconds")

Average latency: 0.0027351874003215928 seconds


/var/folders/2z/b4b21nyd0jxcp1kh766nw8hm0000gp/T/ipykernel_35202/1274102172.py:5: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [19]:
table = db["cats_and_dogs"]

In [20]:
len(table)

4

Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [21]:
table.delete("species='dog'")

DeleteResult(version=4)

In [22]:
len(table)

2

## What if I messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [23]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 23, 13, 73999),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 26, 25, 916323),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 26, 55, 331408),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 8, 22, 0, 2, 46, 687355),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [24]:
table = db["cats_and_dogs"]

In [25]:
len(table)

2

In [26]:
# restore to version 3
table.restore(3)

In [27]:
# delete the dogs instead

len(table)

4

In [28]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 23, 13, 73999),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 26, 25, 916323),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 8, 21, 23, 26, 55, 331408),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 8, 22, 0, 2, 46, 687355),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2025, 8, 22, 0, 3, 30, 417187),
  'metadata': {}}]

In [29]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [30]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [32]:
db.drop_table("cats_and_dogs", ignore_missing=True)

How would you verify that the table has indeed been deleted?

In [33]:
table.name in db

False

## Summary

Congrats, in this exercise you've learned the basic operations of vector databases from creating tables, to adding data, and to querying the data. You've learned how to create indices and you saw first hand how it changes the performance and the accuracy. Lastly, you've learned how to debug and rollback when errors happen.